In [1]:
"modules that needed to be loaded from the main project"
import sys
sys.path.append('/home/harish/GitRepos/Smart_Helm/')

In [2]:
"""packages to import all the required packages"""
import pandas as pd
import swifter
import os
from data_processing.raw_data_parser import transform_eeg
from data_processing.raw_data_parser import transform_etd
from data_processing.raw_data_parser import transform_gps_xdf
from data_processing.raw_data_parser import get_experiment_details
from data_processing.fact_prep import create_fact_df
from data_processing.fact_prep import fact_df_main
import pyxdf
import dtale

In [29]:
filepath = "test_files/Rytle_2021_11_05_BS_02_raw_eeg_sub-P001_ses-S001_task-Default_run-001_eeg.xdf"
remote_file = "Rytle/2021/10/07/KZ/02"
dataset_rytle, header = pyxdf.load_xdf(filepath)

In [5]:
header

{'info': defaultdict(list,
             {'version': ['1.0'], 'datetime': ['2021-10-07T13:35:48+0200']})}

In [32]:
dataset_rytle[0]["time_stamps"][1000]

620067.097090721

In [5]:
etd, exp_date = transform_etd(filepath, remote_file)
eeg = transform_eeg(filepath, exp_date, remote_file)
gps = transform_gps_xdf(filepath, exp_date, remote_file)

In [7]:
gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   altitude                0 non-null      float64
 1   latitude                0 non-null      float64
 2   longitude               0 non-null      float64
 3   accuracy                0 non-null      float64
 4   velocity                0 non-null      float64
 5   lsl_timestamp           0 non-null      object 
 6   participant_id          1 non-null      object 
 7   experiment_nr           1 non-null      int64  
 8   experiment_location_id  1 non-null      object 
 9   experiment_date         1 non-null      object 
dtypes: float64(5), int64(1), object(4)
memory usage: 208.0+ bytes


### TODO 1st step: first join etd and gps data
### TODO 2nd : then use the joined df to formulate the corresponding DF
### TODO 3rd step: join etd_gps_joined dataframe with the eeg
### TODO 4th : __similarly__ formulate the main df

In [ ]:
def get_gps_lsl_tst_join(ts_etd, gps_df):
    df = gps_df.loc[(gps_df["lsl_timestamp"] <= ts_etd) & (ts_etd < gps_df["next_lsl_timestamp"])]
    if df.empty:
        return gps_df.loc[gps_df["lsl_timestamp"] == gps_df["lsl_timestamp"].max()]["lsl_timestamp"].values[0]
    else:
        return df["lsl_timestamp"].values[0]

In [ ]:
gps = gps.sort_values(by = "lsl_timestamp")
gps["next_lsl_timestamp"] = gps["lsl_timestamp"].shift(-1)
etd["joinable_timestamp_with_gps"] = etd.apply(lambda row: get_gps_lsl_tst_join(row["lsl_timestamp"], gps), axis=1)
cols_to_select_from_gps_df = [
    "lsl_timestamp",
    "altitude",
    'latitude', 
    "longitude",
     "accuracy",
    "velocity"
]

etd_joined_gps_df = etd.merge(
    gps[cols_to_select_from_gps_df], 
    left_on="joinable_timestamp_with_gps", 
    right_on="lsl_timestamp", 
    how="left",
    suffixes = ("_etd", "_gps")
)
etd_joined_gps_df = etd_joined_gps_df.drop(columns=["lsl_timestamp_gps", "joinable_timestamp_with_gps"])
etd_joined_gps_df = etd_joined_gps_df.rename(columns = {'lsl_timestamp_etd' : "lsl_timestamp"})

### from here forming the  one big table

In [13]:
def get_etd_gps_joinable_tst(eeg_tst, etd_joined_gps):
    df = etd_joined_gps.loc[(etd_joined_gps["lsl_timestamp"] <= eeg_tst) & (etd_joined_gps["next_lsl_timestamp"] > eeg_tst)]
    if df.empty:
        return etd_joined_gps.loc[etd_joined_gps["lsl_timestamp"] == etd_joined_gps["lsl_timestamp"].max()]["lsl_timestamp"].values[0]
    else: 
        return df["lsl_timestamp"].values[0]


In [16]:
etd_joined_gps_df = etd_joined_gps_df.sort_values(by = "lsl_timestamp")
etd_joined_gps_df["next_lsl_timestamp"] = etd_joined_gps_df["lsl_timestamp"].shift(-1)
eeg["eeg_joinable_column_with_etd_gps"] = eeg.apply(lambda row: get_etd_gps_joinable_tst(row["lsl_timestamp"], etd_joined_gps_df), axis = 1)
etd_gps_columns = [
    "eye_gaze_origin_x",
    "eye_gaze_origin_y",
    "eye_gaze_origin_z",
    "eye_gaze_x",
    "eye_gaze_y",
    "eye_gaze_z",
    "altitude",
    "lsl_timestamp",
    'latitude', 
    "longitude",
     "accuracy",
    "velocity"
    
]
eeg_etd_gps = eeg.merge(
     etd_joined_gps_df[etd_gps_columns], 
    left_on="eeg_joinable_column_with_etd_gps", 
    right_on="lsl_timestamp", 
    how="left",
    suffixes = ("_eeg", "_etd_gps")
)
eeg_etd_gps = eeg_etd_gps.drop(columns= ["eeg_joinable_column_with_etd_gps", "lsl_timestamp_etd_gps"])
eeg_etd_gps = eeg_etd_gps.rename(columns = {"lsl_timestamp_eeg": "lsl_timestamp" })

In [7]:
""" merging etd and eeg """
etd = etd.sort_values(by = "lsl_timestamp")
etd["next_lsl_timestamp"] = etd["lsl_timestamp"].shift(-1)


In [8]:
def get_etd_lsl_tst_join(ts_eeg, etd_df):
    df = etd_df.loc[(etd_df["lsl_timestamp"] <= ts_eeg) & (ts_eeg < etd_df["next_lsl_timestamp"])]
    if df.empty:
        return etd_df.loc[etd_df["lsl_timestamp"] == etd_df["lsl_timestamp"].max()]["lsl_timestamp"].values[0]
    else:
        return df["lsl_timestamp"].values[0]

In [9]:
eeg["joinable_timestamp_with_etd"] = eeg.apply(lambda row: get_etd_lsl_tst_join(row["lsl_timestamp"], etd), axis=1)

In [12]:
cols_to_select_from_etd_df = [
    "lsl_timestamp",
    "eye_gaze_origin_x",
    "eye_gaze_origin_y",
    "eye_gaze_origin_z",
    "eye_gaze_x",
    "eye_gaze_y",
    "eye_gaze_z",
]

eeg_joined_etd_df = eeg.merge(
    etd[cols_to_select_from_etd_df], 
    left_on="joinable_timestamp_with_etd", 
    right_on="lsl_timestamp", 
    how="left",
    suffixes = ("_eeg", "_etd")
)
eeg_joined_etd_df = eeg_joined_etd_df.drop(columns=["lsl_timestamp_etd", "joinable_timestamp_with_etd"])
eeg_joined_etd_df = eeg_joined_etd_df.rename(columns = {'lsl_timestamp_eeg' : "lsl_timestamp"})

In [13]:
eeg_joined_etd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193256 entries, 0 to 193255
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lsl_timestamp           193256 non-null  float64
 1   eeg_value_1             193256 non-null  float32
 2   eeg_value_2             193256 non-null  float32
 3   eeg_value_3             193256 non-null  float32
 4   eeg_value_4             193256 non-null  float32
 5   eeg_value_5             193256 non-null  float32
 6   eeg_value_6             193256 non-null  float32
 7   eeg_value_7             193256 non-null  float32
 8   experiment_date         193256 non-null  object 
 9   participant_id          193256 non-null  object 
 10  experiment_nr           193256 non-null  int64  
 11  experiment_location_id  193256 non-null  object 
 12  eye_gaze_origin_x       167240 non-null  float64
 13  eye_gaze_origin_y       167240 non-null  float64
 14  eye_gaze_origin_z   

In [14]:
eeg_joined_etd_df['lat'] = 

In [17]:
eeg_joined_etd_df.head()

,lsl_timestamp,eeg_value_1,eeg_value_2,eeg_value_3,eeg_value_4,eeg_value_5,eeg_value_6,eeg_value_7,experiment_date,participant_id,experiment_nr,experiment_location_id,eye_gaze_origin_x,eye_gaze_origin_y,eye_gaze_origin_z,eye_gaze_x,eye_gaze_y,eye_gaze_z,lat
0,9042.991083,4.441718,-9.139711,-2.137319,1.556978,-40.239429,-20.978626,-15.485593,2021-10-05,KZ,2,Rytle,-0.229819,-0.177579,-0.956896,-46.55487,-9.13602,-509.8745,None
1,9042.995083,8.273370,-4.722237,-1.185792,-9.840579,-37.528236,-14.298180,-22.497044,2021-10-05,KZ,2,Rytle,-0.229819,-0.177579,-0.956896,-46.55487,-9.13602,-509.8745,None
2,9042.999083,14.155184,4.059216,2.347409,-12.991497,-32.086842,-5.399246,-27.876507,2021-10-05,KZ,2,Rytle,-0.229819,-0.177579,-0.956896,-46.55487,-9.13602,-509.8745,None
3,9043.003083,20.673042,15.660712,7.396507,-6.706311,-24.661154,4.322580,-30.333467,2021-10-05,KZ,2,Rytle,-0.229819,-0.177579,-0.956896,-46.55487,-9.13602,-509.8745,None
4,9043.007083,24.219883,25.383415,10.560529,4.460753,-18.712088,11.537330,-30.614294,2021-10-05,KZ,2,Rytle,-0.229819,-0.177579,-0.956896,-46.55487,-9.13602,-509.8745,None


In [22]:
""" explaratory data analysis"""
df_vis = dtale.show(eeg_etd_gps)
df_vis.open_browser()

In [21]:
def create_fact_df(etd: pd.DataFrame, eeg: pd.DataFrame, gps: pd.DataFrame ) -> pd.DataFrame:
    """

    Args:
        etd:
        eeg:
        gps:

    Returns:

    """


    return None
#  Step1: insert gps data with one row dataframe where gps data is missing
# step2: similarly in the fact data also insert the 4 constraints at  the end not at the beginning